In [33]:
from local_vars import test_folder
import os 
import numpy as np

data_folder = "SkullScan2_Images"
input_sequence_name = "SkullScan2"
image_name = "Image_Image"
data_name = "segmentations.npy"
output_markups_name = "F"
minimum_distance = 10 #in millimeters
segmentation_threshold = 0.8

In [34]:
data_fullpath = os.path.join(test_folder, data_folder)
segmentations_fullname = os.path.join(data_fullpath, data_name)

segmentations = np.load(segmentations_fullname)
print(data_folder)

SkullScan2_Images


In [35]:
input_sequence = slicer.util.getFirstNodeByName(input_sequence_name, className="vtkMRMLSequenceBrowserNode")
image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
transform_node = slicer.util.getFirstNodeByName("ImageToReference", className="vtkMRMLTransformNode")
output_markups = slicer.util.getFirstNodeByName(output_markups_name, className="vtkMRMLMarkupsFiducialNode")

In [47]:
input_image_size = segmentations.shape[1]
n = input_sequence.GetNumberOfItems()

#based on assumption that the US image is square
bounds = [0,0,0,0,0,0]
image_node.GetBounds(bounds)
us_image_size = bounds[1]-bounds[0]

step_size = input_image_size / us_image_size

point_list = []

for i in range(n):
    if  sum(sum(segmentations[i])) >= segmentation_threshold:#check to ensure there is at least one valid possible point
        testpoint = np.matrix([[500],[177],[0],[1]])

        point = np.matrix([[0],[0],[0],[1]])

        transform = transform_node.GetMatrixTransformToParent()
        transform_matrix = np.matrix([[transform.GetElement(0,0),transform.GetElement(0,1),transform.GetElement(0,2),transform.GetElement(0,3)],
                                     [transform.GetElement(1,0),transform.GetElement(1,1),transform.GetElement(1,2),transform.GetElement(1,3)],
                                     [transform.GetElement(2,0),transform.GetElement(2,1),transform.GetElement(2,2),transform.GetElement(2,3)],
                                     [transform.GetElement(3,0),transform.GetElement(3,1),transform.GetElement(3,2),transform.GetElement(3,3)]])

        previous_point = []

        for j in range(input_image_size):
            
            max_indicies = np.where(segmentations[i,:,j] == max(segmentations[i,:,j]))[0]
            max_index = max_indicies[-1]
            
            if segmentations[i,max_index,j] >= segmentation_threshold:                
                point[0] = j*step_size

                #need to check for proper us ultrasound orientation?
                point[1] = 511 - max_index*step_size #this is for if images are upside down

                transformed_point = np.dot(transform_matrix,point)

                if previous_point == []:
                    previous_point = transformed_point
                    point_list.append(transformed_point) 
                else:
                    dist = np.linalg.norm(transformed_point-previous_point)

                    if dist >= minimum_distance:
                        point_list.append(transformed_point)
                        previous_point = transformed_point            

    input_sequence.SelectNextItem()  
    
print(len(point_list))

5348


In [42]:
#for testing purposes
for point in point_list:
    output_markups.AddFiducial(point[0],point[1],point[2])

In [48]:
#np.random.shuffle(point_list)

marker_points = []

marker_points.append(point_list[0])
num_marker_points = 1

for target_point in point_list[1:]:
    too_close = False
    marker_point_index = 0
    
    while not(too_close) and marker_point_index < (num_marker_points):
        dist = np.linalg.norm(target_point-marker_points[marker_point_index])

        if dist < minimum_distance:
            too_close = True
        marker_point_index += 1
            
    if not(too_close):
        marker_points.append(target_point)
        num_marker_points += 1
        
print(num_marker_points)        

25


In [45]:
output_markups = slicer.util.getFirstNodeByName(output_markups_name, className="vtkMRMLMarkupsFiducialNode")

for point in marker_points:
    output_markups.AddFiducial(point[0],point[1],point[2])